In [1]:
pip install tableau-api-lib

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd

In [3]:
from tableau_api_lib import TableauServerConnection
from tableau_api_lib.sample import sample_config
from tableau_api_lib.utils import querying, flatten_dict_column, flatten_dict_list_column

In [7]:
# sample config
tableau_server_config = {
  'tableau_oline': {
    'server': 'https://10ax.online.tableau.com',
    'api_version': '3.17',
    'personal_access_token_name': 'asmaa',
    'personal_access_token_secret':'kxyuaQRhR7OaJnsXM4uyNQ==:O6rgV7bMUceIXKhXP10LDszghCLtKndf',
    'site_name': 'asmaa',
    'site_url': 'asmaa'
  }
}

In [8]:
conn=TableauServerConnection(tableau_server_config ,env='tableau_oline')

In [9]:
conn.sign_in()

<Response [200]>

In [10]:
workbook_permissions_df = pd.DataFrame()


In [14]:
workbooks_df = querying.get_workbooks_dataframe(conn)[["project", "name", "id"]]


In [15]:
workbooks_df

,project,name,id
0,"{'id': '385b3746-0eb5-4993-abba-c89477ed175c',...",Regional,fb811dfe-51b2-4186-954b-328f8d2cca50
1,"{'id': '385b3746-0eb5-4993-abba-c89477ed175c',...",Superstore,1fc1448c-91e0-4546-8cda-6eb336c345d4


In [10]:
workbooks_df = flatten_dict_column(df=workbooks_df, keys=["id", "name"], col_name="project")


In [11]:
workbooks_df

,name,id,project_id,project_name
0,Regional,fb811dfe-51b2-4186-954b-328f8d2cca50,385b3746-0eb5-4993-abba-c89477ed175c,Samples
1,Superstore,1fc1448c-91e0-4546-8cda-6eb336c345d4,385b3746-0eb5-4993-abba-c89477ed175c,Samples


In [12]:
workbooks_df.rename(columns={"id": "workbook_id", "name": "workbook_name"}, inplace=True)

In [13]:
workbooks_df

,workbook_name,workbook_id,project_id,project_name
0,Regional,fb811dfe-51b2-4186-954b-328f8d2cca50,385b3746-0eb5-4993-abba-c89477ed175c,Samples
1,Superstore,1fc1448c-91e0-4546-8cda-6eb336c345d4,385b3746-0eb5-4993-abba-c89477ed175c,Samples


In [14]:
all_group_users_df = pd.DataFrame()
groups_df = querying.get_groups_dataframe(conn)

In [15]:
groups_df

,domain,id,name
0,local,db49c95a-8278-4e50-87fd-ed4aff66e8c0,All Users


In [16]:
for index, group in groups_df.iterrows():
    print(f"joining users from group '{group.id}'")
    group_users_df = querying.get_group_users_dataframe(conn=conn, group_id=group.id)
    group_users_df.rename(columns={"name": "user_name", "id": "user_id"}, inplace=True)
    group_users_df["group_id"] = group["id"]
    group_users_df["group_name"] = group["name"]
    group_users_df["domain"] = group["domain"]
    all_group_users_df = pd.concat([all_group_users_df, group_users_df])


joining users from group 'db49c95a-8278-4e50-87fd-ed4aff66e8c0'


In [17]:
 all_group_users_df 

,domain,authSetting,email,externalAuthUserId,fullName,user_id,lastLogin,user_name,siteRole,locale,language,group_id,group_name
0,local,ServerDefault,asmaa.ali.kashef@gmail.com,1463670232-8efeaab9c0974ceca78fb7b5bb88dbaf,asmaa kashef,34252609-7bd8-4365-88de-af4e5757a175,2022-09-01T21:23:43Z,asmaa.ali.kashef@gmail.com,SiteAdministratorCreator,,en,db49c95a-8278-4e50-87fd-ed4aff66e8c0,All Users


In [18]:
# group permissions
for _, workbook in workbooks_df.iterrows():
    print(f"fetching group permissions for workbook '{workbook.workbook_name}...'")
    response = conn.query_workbook_permissions(workbook_id=workbook.workbook_id)
    try:
        permissions_df = pd.DataFrame(response.json()["permissions"]["granteeCapabilities"])
    except KeyError:
        print(f"skipping workbook '{workbook.workbook_name}' because no permissions rules are defined for it...")
        continue
    if "group" not in permissions_df.columns:
        continue
    permissions_df = permissions_df[permissions_df["group"].notnull()]
    permissions_df = flatten_dict_column(df=permissions_df, keys=["id"], col_name="group")
    permissions_df = flatten_dict_list_column(df=permissions_df, col_name="capabilities")
    permissions_df = flatten_dict_column(df=permissions_df, keys=["name", "mode"], col_name="capability")
    permissions_df["workbook_id"] = workbook.workbook_id
    permissions_df["workbook_name"] = workbook.workbook_name
    permissions_df["project_name"] = workbook.project_name
    permissions_df["project_id"] = workbook.project_id
    permissions_df["join_col"] = 1

    workbook_permissions_df = pd.concat(
        [workbook_permissions_df, permissions_df.merge(all_group_users_df, on=["group_id"])]
    )


fetching group permissions for workbook 'Regional...'
fetching group permissions for workbook 'Superstore...'


In [19]:
   workbook_permissions_df 

,group_id,capability_name,capability_mode,workbook_id,workbook_name,project_name,project_id,join_col,domain,authSetting,email,externalAuthUserId,fullName,user_id,lastLogin,user_name,siteRole,locale,language,group_name
0,db49c95a-8278-4e50-87fd-ed4aff66e8c0,ViewComments,Allow,fb811dfe-51b2-4186-954b-328f8d2cca50,Regional,Samples,385b3746-0eb5-4993-abba-c89477ed175c,1,local,ServerDefault,asmaa.ali.kashef@gmail.com,1463670232-8efeaab9c0974ceca78fb7b5bb88dbaf,asmaa kashef,34252609-7bd8-4365-88de-af4e5757a175,2022-09-01T21:23:43Z,asmaa.ali.kashef@gmail.com,SiteAdministratorCreator,,en,All Users
1,db49c95a-8278-4e50-87fd-ed4aff66e8c0,Filter,Allow,fb811dfe-51b2-4186-954b-328f8d2cca50,Regional,Samples,385b3746-0eb5-4993-abba-c89477ed175c,1,local,ServerDefault,asmaa.ali.kashef@gmail.com,1463670232-8efeaab9c0974ceca78fb7b5bb88dbaf,asmaa kashef,34252609-7bd8-4365-88de-af4e5757a175,2022-09-01T21:23:43Z,asmaa.ali.kashef@gmail.com,SiteAdministratorCreator,,en,All Users
2,db49c95a-8278-4e50-87fd-ed4aff66e8c0,Write,Allow,fb811dfe-51b2-4186-954b-328f8d2cca50,Regional,Samples,385b3746-0eb5-4993-abba-c89477ed175c,1,local,ServerDefault,asmaa.ali.kashef@gmail.com,1463670232-8efeaab9c0974ceca78fb7b5bb88dbaf,asmaa kashef,34252609-7bd8-4365-88de-af4e5757a175,2022-09-01T21:23:43Z,asmaa.ali.kashef@gmail.com,SiteAdministratorCreator,,en,All Users
3,db49c95a-8278-4e50-87fd-ed4aff66e8c0,ExportData,Allow,fb811dfe-51b2-4186-954b-328f8d2cca50,Regional,Samples,385b3746-0eb5-4993-abba-c89477ed175c,1,local,ServerDefault,asmaa.ali.kashef@gmail.com,1463670232-8efeaab9c0974ceca78fb7b5bb88dbaf,asmaa kashef,34252609-7bd8-4365-88de-af4e5757a175,2022-09-01T21:23:43Z,asmaa.ali.kashef@gmail.com,SiteAdministratorCreator,,en,All Users
4,db49c95a-8278-4e50-87fd-ed4aff66e8c0,ShareView,Allow,fb811dfe-51b2-4186-954b-328f8d2cca50,Regional,Samples,385b3746-0eb5-4993-abba-c89477ed175c,1,local,ServerDefault,asmaa.ali.kashef@gmail.com,1463670232-8efeaab9c0974ceca78fb7b5bb88dbaf,asmaa kashef,34252609-7bd8-4365-88de-af4e5757a175,2022-09-01T21:23:43Z,asmaa.ali.kashef@gmail.com,SiteAdministratorCreator,,en,All Users
5,db49c95a-8278-4e50-87fd-ed4aff66e8c0,Read,Allow,fb811dfe-51b2-4186-954b-328f8d2cca50,Regional,Samples,385b3746-0eb5-4993-abba-c89477ed175c,1,local,ServerDefault,asmaa.ali.kashef@gmail.com,1463670232-8efeaab9c0974ceca78fb7b5bb88dbaf,asmaa kashef,34252609-7bd8-4365-88de-af4e5757a175,2022-09-01T21:23:43Z,asmaa.ali.kashef@gmail.com,SiteAdministratorCreator,,en,All Users
6,db49c95a-8278-4e50-87fd-ed4aff66e8c0,AddComment,Allow,fb811dfe-51b2-4186-954b-328f8d2cca50,Regional,Samples,385b3746-0eb5-4993-abba-c89477ed175c,1,local,ServerDefault,asmaa.ali.kashef@gmail.com,1463670232-8efeaab9c0974ceca78fb7b5bb88dbaf,asmaa kashef,34252609-7bd8-4365-88de-af4e5757a175,2022-09-01T21:23:43Z,asmaa.ali.kashef@gmail.com,SiteAdministratorCreator,,en,All Users
7,db49c95a-8278-4e50-87fd-ed4aff66e8c0,ViewUnderlyingData,Allow,fb811dfe-51b2-4186-954b-328f8d2cca50,Regional,Samples,385b3746-0eb5-4993-abba-c89477ed175c,1,local,ServerDefault,asmaa.ali.kashef@gmail.com,1463670232-8efeaab9c0974ceca78fb7b5bb88dbaf,asmaa kashef,34252609-7bd8-4365-88de-af4e5757a175,2022-09-01T21:23:43Z,asmaa.ali.kashef@gmail.com,SiteAdministratorCreator,,en,All Users
8,db49c95a-8278-4e50-87fd-ed4aff66e8c0,ExportImage,Allow,fb811dfe-51b2-4186-954b-328f8d2cca50,Regional,Samples,385b3746-0eb5-4993-abba-c89477ed175c,1,local,ServerDefault,asmaa.ali.kashef@gmail.com,1463670232-8efeaab9c0974ceca78fb7b5bb88dbaf,asmaa kashef,34252609-7bd8-4365-88de-af4e5757a175,2022-09-01T21:23:43Z,asmaa.ali.kashef@gmail.com,SiteAdministratorCreator,,en,All Users
0,db49c95a-8278-4e50-87fd-ed4aff66e8c0,ViewComments,Allow,1fc1448c-91e0-4546-8cda-6eb336c345d4,Superstore,Samples,385b3746-0eb5-4993-abba-c89477ed175c,1,local,ServerDefault,asmaa.ali.kashef@gmail.com,1463670232-8efeaab9c0974ceca78fb7b5bb88dbaf,asmaa kashef,34252609-7bd8-4365-88de-af4e5757a175,2022-09-01T21:23:43Z,asmaa.ali.kashef@gmail.com,SiteAdministratorCreator,,en,All Users


In [20]:
# user permissions
users_df = querying.get_users_dataframe(conn)
users_df.rename(columns={"id": "user_id", "name": "user_name"}, inplace=True)

for _, workbook in workbooks_df.iterrows():
    print(f"fetching user permissions for workbook '{workbook.workbook_name}...'")
    response = conn.query_workbook_permissions(workbook_id=workbook.workbook_id)
    try:
        permissions_df = pd.DataFrame(response.json()["permissions"]["granteeCapabilities"])
    except KeyError:
        print(f"skipping workbook '{workbook.workbook_name}' because no permissions rules are defined for it...")
        continue
    if "user" not in permissions_df.columns:
        continue
    permissions_df = permissions_df[permissions_df["user"].notnull()]
    permissions_df = flatten_dict_column(df=permissions_df, keys=["id"], col_name="user")
    permissions_df = flatten_dict_list_column(df=permissions_df, col_name="capabilities")
    permissions_df = flatten_dict_column(df=permissions_df, keys=["name", "mode"], col_name="capability")
    permissions_df["workbook_id"] = workbook.workbook_id
    permissions_df["workbook_name"] = workbook.workbook_name
    permissions_df["project_name"] = workbook.project_name
    permissions_df["project_id"] = workbook.project_id
    permissions_df["join_col"] = 1

    workbook_permissions_df = pd.concat(
        [workbook_permissions_df, permissions_df.merge(users_df, on=["user_id"])]
    )

workbook_permissions_df.to_csv("workbook_permissions.csv", sep=",", header=True, index=False)

fetching user permissions for workbook 'Regional...'
fetching user permissions for workbook 'Superstore...'


In [21]:
workbook_permissions_df

,group_id,capability_name,capability_mode,workbook_id,workbook_name,project_name,project_id,join_col,domain,authSetting,email,externalAuthUserId,fullName,user_id,lastLogin,user_name,siteRole,locale,language,group_name
0,db49c95a-8278-4e50-87fd-ed4aff66e8c0,ViewComments,Allow,fb811dfe-51b2-4186-954b-328f8d2cca50,Regional,Samples,385b3746-0eb5-4993-abba-c89477ed175c,1,local,ServerDefault,asmaa.ali.kashef@gmail.com,1463670232-8efeaab9c0974ceca78fb7b5bb88dbaf,asmaa kashef,34252609-7bd8-4365-88de-af4e5757a175,2022-09-01T21:23:43Z,asmaa.ali.kashef@gmail.com,SiteAdministratorCreator,,en,All Users
1,db49c95a-8278-4e50-87fd-ed4aff66e8c0,Filter,Allow,fb811dfe-51b2-4186-954b-328f8d2cca50,Regional,Samples,385b3746-0eb5-4993-abba-c89477ed175c,1,local,ServerDefault,asmaa.ali.kashef@gmail.com,1463670232-8efeaab9c0974ceca78fb7b5bb88dbaf,asmaa kashef,34252609-7bd8-4365-88de-af4e5757a175,2022-09-01T21:23:43Z,asmaa.ali.kashef@gmail.com,SiteAdministratorCreator,,en,All Users
2,db49c95a-8278-4e50-87fd-ed4aff66e8c0,Write,Allow,fb811dfe-51b2-4186-954b-328f8d2cca50,Regional,Samples,385b3746-0eb5-4993-abba-c89477ed175c,1,local,ServerDefault,asmaa.ali.kashef@gmail.com,1463670232-8efeaab9c0974ceca78fb7b5bb88dbaf,asmaa kashef,34252609-7bd8-4365-88de-af4e5757a175,2022-09-01T21:23:43Z,asmaa.ali.kashef@gmail.com,SiteAdministratorCreator,,en,All Users
3,db49c95a-8278-4e50-87fd-ed4aff66e8c0,ExportData,Allow,fb811dfe-51b2-4186-954b-328f8d2cca50,Regional,Samples,385b3746-0eb5-4993-abba-c89477ed175c,1,local,ServerDefault,asmaa.ali.kashef@gmail.com,1463670232-8efeaab9c0974ceca78fb7b5bb88dbaf,asmaa kashef,34252609-7bd8-4365-88de-af4e5757a175,2022-09-01T21:23:43Z,asmaa.ali.kashef@gmail.com,SiteAdministratorCreator,,en,All Users
4,db49c95a-8278-4e50-87fd-ed4aff66e8c0,ShareView,Allow,fb811dfe-51b2-4186-954b-328f8d2cca50,Regional,Samples,385b3746-0eb5-4993-abba-c89477ed175c,1,local,ServerDefault,asmaa.ali.kashef@gmail.com,1463670232-8efeaab9c0974ceca78fb7b5bb88dbaf,asmaa kashef,34252609-7bd8-4365-88de-af4e5757a175,2022-09-01T21:23:43Z,asmaa.ali.kashef@gmail.com,SiteAdministratorCreator,,en,All Users
5,db49c95a-8278-4e50-87fd-ed4aff66e8c0,Read,Allow,fb811dfe-51b2-4186-954b-328f8d2cca50,Regional,Samples,385b3746-0eb5-4993-abba-c89477ed175c,1,local,ServerDefault,asmaa.ali.kashef@gmail.com,1463670232-8efeaab9c0974ceca78fb7b5bb88dbaf,asmaa kashef,34252609-7bd8-4365-88de-af4e5757a175,2022-09-01T21:23:43Z,asmaa.ali.kashef@gmail.com,SiteAdministratorCreator,,en,All Users
6,db49c95a-8278-4e50-87fd-ed4aff66e8c0,AddComment,Allow,fb811dfe-51b2-4186-954b-328f8d2cca50,Regional,Samples,385b3746-0eb5-4993-abba-c89477ed175c,1,local,ServerDefault,asmaa.ali.kashef@gmail.com,1463670232-8efeaab9c0974ceca78fb7b5bb88dbaf,asmaa kashef,34252609-7bd8-4365-88de-af4e5757a175,2022-09-01T21:23:43Z,asmaa.ali.kashef@gmail.com,SiteAdministratorCreator,,en,All Users
7,db49c95a-8278-4e50-87fd-ed4aff66e8c0,ViewUnderlyingData,Allow,fb811dfe-51b2-4186-954b-328f8d2cca50,Regional,Samples,385b3746-0eb5-4993-abba-c89477ed175c,1,local,ServerDefault,asmaa.ali.kashef@gmail.com,1463670232-8efeaab9c0974ceca78fb7b5bb88dbaf,asmaa kashef,34252609-7bd8-4365-88de-af4e5757a175,2022-09-01T21:23:43Z,asmaa.ali.kashef@gmail.com,SiteAdministratorCreator,,en,All Users
8,db49c95a-8278-4e50-87fd-ed4aff66e8c0,ExportImage,Allow,fb811dfe-51b2-4186-954b-328f8d2cca50,Regional,Samples,385b3746-0eb5-4993-abba-c89477ed175c,1,local,ServerDefault,asmaa.ali.kashef@gmail.com,1463670232-8efeaab9c0974ceca78fb7b5bb88dbaf,asmaa kashef,34252609-7bd8-4365-88de-af4e5757a175,2022-09-01T21:23:43Z,asmaa.ali.kashef@gmail.com,SiteAdministratorCreator,,en,All Users
0,db49c95a-8278-4e50-87fd-ed4aff66e8c0,ViewComments,Allow,1fc1448c-91e0-4546-8cda-6eb336c345d4,Superstore,Samples,385b3746-0eb5-4993-abba-c89477ed175c,1,local,ServerDefault,asmaa.ali.kashef@gmail.com,1463670232-8efeaab9c0974ceca78fb7b5bb88dbaf,asmaa kashef,34252609-7bd8-4365-88de-af4e5757a175,2022-09-01T21:23:43Z,asmaa.ali.kashef@gmail.com,SiteAdministratorCreator,,en,All Users
